In [ ]:
!nvidia-smi

Thu Nov 21 05:14:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Get content

In [ ]:
!pip install faiss-cpu -q
!pip install beautifulsoup4 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 52.6 MB/s eta 0:00:00


In [ ]:
from bs4 import BeautifulSoup
import requests

query = "What is a RAG system?"
query = query.replace(' ', '+')

url = f"https://www.google.com/search?q={query}"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    urls = []
    gfg = 0
    for g in soup.find_all('a'):
        href = g.get('href')
        if href and "/url?esrc=s&q=&rct=j&sa=U&url=" in href:
            link = href.split("/url?esrc=s&q=&rct=j&sa=U&url=")[1].split("&")[0]
            #print(link)
            #if (link.split("https://")[1].split(".")[0] != "scholar"):
            urls.append(link)
else:
    print("Failed to fetch results")



In [ ]:
len(urls)

17

In [ ]:
urls

['https://www.cohesity.com/glossary/retrieval-augmented-generation-rag/',
 'https://aws.amazon.com/what-is/retrieval-augmented-generation/',
 'https://cloud.google.com/use-cases/retrieval-augmented-generation',
 'https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/',
 'https://www.oracle.com/artificial-intelligence/generative-ai/retrieval-augmented-generation-rag/',
 'https://www.oracle.com/in/artificial-intelligence/generative-ai/retrieval-augmented-generation-rag/',
 'https://www.oracle.com/uk/artificial-intelligence/generative-ai/retrieval-augmented-generation-rag/',
 'https://www.oracle.com/au/artificial-intelligence/generative-ai/retrieval-augmented-generation-rag/',
 'https://www.oracle.com/ie/artificial-intelligence/generative-ai/retrieval-augmented-generation-rag/',
 'https://www.datacamp.com/blog/what-is-retrieval-augmented-generation-rag',
 'https://www.datacamp.com/blog/what-is-retrieval-augmented-generation-rag%23what-is-rag%3F-rag,o',
 'https://www.datacam

In [ ]:
texts = []
final_urls = []
total = 0
for url in urls:
    success = False
    while not success:
        try:
            response = requests.get(url, headers=headers, timeout=10)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')

                for tag in soup(['script', 'style', 'header', 'footer', 'nav']):
                    tag.decompose()

                page_text = ' '.join(tag.get_text() for tag in soup.find_all(['p', 'h1', 'h2', 'h3', 'li']))
                texts.append(page_text)
                success = True
                total += 1
                final_urls.append(url)
            else:
                print(f"Failed to fetch {url}: {response.status_code}")
                break
        except Exception as e:
            print(f"Error fetching {url}: {e}")
            break

    if (total == 3):
        break


In [ ]:
final_urls

['https://www.cohesity.com/glossary/retrieval-augmented-generation-rag/',
 'https://aws.amazon.com/what-is/retrieval-augmented-generation/',
 'https://cloud.google.com/use-cases/retrieval-augmented-generation']

In [ ]:
texts

["Cohesity named a Leader again! 2023 Gartner® Magic Quadrant™ for Enterprise Backup and Recovery Software Solutions Glossary Retrieval augmented generation (RAG) Retrieval augmented generation (RAG) What is retrieval augmented generation (RAG)? Retrieval augmented generation (RAG) is a natural language processing (NLP) technique that combines the strengths of both retrieval- and generative-based artificial intelligence (AI) models. RAG AI can deliver accurate results that make the most of pre-existing knowledge but can also process and consolidate that knowledge to create unique, context-aware answers, instructions, or explanations in human-like language rather than just summarizing the retrieved data. RAG AI differs from generative AI in that it is a superset of generative AI. RAG combines the strengths of both generative AI and retrieval AI. RAG is also different from cognitive AI, which mimics the way the human brain works to get its results. How does retrieval augmented generation

In [ ]:
len(texts)

3

In [ ]:
len(texts[0]), len(texts[1]), len(texts[2])

(8422, 10109, 7083)

# Cleaning/Preprocessing

In [ ]:
import re

def clean_text(text):
    text = re.sub(r"\[.*?\]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

texts[0] = clean_text(texts[0])
texts[1] = clean_text(texts[1])
texts[2] = clean_text(texts[2])

In [ ]:
texts

["Cohesity named a Leader again! 2023 Gartner® Magic Quadrant™ for Enterprise Backup and Recovery Software Solutions Glossary Retrieval augmented generation (RAG) Retrieval augmented generation (RAG) What is retrieval augmented generation (RAG)? Retrieval augmented generation (RAG) is a natural language processing (NLP) technique that combines the strengths of both retrieval- and generative-based artificial intelligence (AI) models. RAG AI can deliver accurate results that make the most of pre-existing knowledge but can also process and consolidate that knowledge to create unique, context-aware answers, instructions, or explanations in human-like language rather than just summarizing the retrieved data. RAG AI differs from generative AI in that it is a superset of generative AI. RAG combines the strengths of both generative AI and retrieval AI. RAG is also different from cognitive AI, which mimics the way the human brain works to get its results. How does retrieval augmented generation

In [ ]:
len(texts[0]), len(texts[1]), len(texts[2])

(8413, 10105, 7081)

In [ ]:
final_text = " ".join(texts)
final_text

'Cohesity named a Leader again! 2023 Gartner® Magic Quadrant™ for Enterprise Backup and Recovery Software Solutions Glossary Retrieval augmented generation (RAG) Retrieval augmented generation (RAG) What is retrieval augmented generation (RAG)? Retrieval augmented generation (RAG) is a natural language processing (NLP) technique that combines the strengths of both retrieval- and generative-based artificial intelligence (AI) models. RAG AI can deliver accurate results that make the most of pre-existing knowledge but can also process and consolidate that knowledge to create unique, context-aware answers, instructions, or explanations in human-like language rather than just summarizing the retrieved data. RAG AI differs from generative AI in that it is a superset of generative AI. RAG combines the strengths of both generative AI and retrieval AI. RAG is also different from cognitive AI, which mimics the way the human brain works to get its results. How does retrieval augmented generation 

In [ ]:
len(final_text)

25601

# Knowledge base using sentence embenddings

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss # Facebook AI Similarity Seacrh
import numpy as np

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
chunks = [final_text[i:i+1500] for i in range(0, len(final_text), 1500)] # chunking
embeddings = model.encode(chunks) # embeddings for each chunk
# embeddings array of the form (N,D) where N = no. of chunks and D = dim of embeddings vec

In [ ]:
dimension = embeddings.shape[1] # extract D
index = faiss.IndexFlatL2(dimension) # initialize L2 eucidean distance for similarity b/w vecs; saying it has dimension number of dimensions for the vectors
index.add(np.array(embeddings)) # add embeddings to FAISS index; FAISS build internal struct for optimal search
print(f"Indexed {len(chunks)} chunks.")


Indexed 18 chunks.


# RAG pipeline

## Retrieval (3 nearest clusters using similarity search)

In [ ]:
user_query = query

query_embedding = model.encode([user_query]) # embeddings for query
D, I = index.search(query_embedding, k=3)  # top 10 closest

relevant_passages = [chunks[i] for i in I[0]]

In [ ]:
relevant_passages_str = ''.join(relevant_passages)

In [ ]:
relevant_passages_str

" pre-trained LLM. This integration enhances the LLM's context, providing it with a more comprehensive understanding of the topic. This augmented context enables the LLM to generate more precise, informative, and engaging responses. Why Use RAG? RAG offers several advantages augmenting traditional methods of text generation, especially when dealing with factual information or data-driven responses. Here are some key reasons why using RAG can be beneficial: Access to fresh information LLMs are limited to their pre-trained data. This leads to outdated and potentially inaccurate responses. RAG overcomes this by providing up-to-date information to LLMs. Factual grounding LLMs are powerful tools for generating creative and engaging text, but they can sometimes struggle with factual accuracy. This is because LLMs are trained on massive amounts of text data, which may contain inaccuracies or biases. Providing “facts” to the LLM as part of the input prompt can mitigate “gen AI hallucinations.”

In [ ]:
len(relevant_passages_str)

4500

## Augmented generation (with RAG)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-Coder-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


messages = [
    {"role": "system", "content": final_text},
    {"role": "user", "content": user_query}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(response)
print("\nSources: \n")
for url in final_urls:
  print(url)

A RAG system is a type of AI system that combines information retrieval and generative artificial intelligence (AI) models to generate human-like responses. RAG systems are designed to provide accurate, relevant, and original responses to user queries. They use pre-existing knowledge sources to provide a context-aware and human-sounding answer to the user's question. RAG systems are often used in fields such as customer support, marketing research, and business intelligence to improve the quality of AI-driven conversations with customers. RAG systems are often used in industries such as healthcare, finance, and manufacturing to provide personalized and accurate responses to user queries. RAG systems are often used in industries such as finance, healthcare, and manufacturing to provide personalized and accurate responses to user queries. RAG systems are often used in industries such as healthcare, finance, and manufacturing to provide personalized and accurate responses to user queries.

## Control data (same prompt without RAG)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name = "Qwen/Qwen2.5-Coder-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": user_query}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(response)


A RAG system is a type of language model that can generate human-like text based on a given instruction. It uses natural language processing techniques to understand and respond to user queries and prompts in a conversational manner.
